Jeongmin Chae | 6022220672 | hw3

In [1]:
import os
from glob import glob
import pandas as pd
pd.set_option('display.max.colwidth', 100)
pd.options.display.precision = 4
import numpy as np
from scipy.stats import bootstrap
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [10]:
class TimeSeriesClassification():
    
    def __init__(self,file_path_dic,tr_file_path_dic,ts_file_path_dic):
        self.file_path_dic=file_path_dic
        self.data=pd.DataFrame()
        self.tr_data=pd.DataFrame()
        self.ts_data=pd.DataFrame()
        self.folder = ['bending1','bending2','cycling','lying','sitting','standing','walking']
        self.columns = ['min1','max1','mean1','50%_1','std1','25%_1','75%_1',
                                                  'min2','max2','mean2','50%_2','std2','25%_2','75%_2',
                                                  'min3','max3','mean3','50%_3','std3','25%_3','75%_3',
                                                  'min4','max4','mean4','50%_4','std4','25%_4','75%_4',
                                                  'min5','max5','mean5','50%_5','std5','25%_5','75%_5',
                                                  'min6','max6','mean6','50%_6','std6','25%_6','75%_6']
        self.tr_file_path_dic=tr_file_path_dic
        self.ts_file_path_dic=ts_file_path_dic
        
        
    
    
    def feature_extraction(self):
        
      
        new_data=[]
        folder = self.folder

        for i in range(len(self.folder)):
            for j in range(len(self.file_path_dic[self.folder[i]])):
                path=self.file_path_dic[self.folder[i]][j]
                
                    
                df = pd.read_csv(path,skiprows=4,usecols=range(1,7))
                df.columns=['a','b','c','d','e','f']
                columns=['a','b','c','d','e','f']
                data=pd.DataFrame()
                for k in range(6):
                    a=df.describe(include='all').loc[['min','max','mean','50%','std','25%','75%']][columns[k]]
                    data=pd.concat([data,a],ignore_index=True)

                new_data.append(np.array(data).reshape(1,len(data)))

        new_data=np.array(new_data).reshape(len(new_data),len(columns)*7)
        data=pd.DataFrame(new_data, columns=self.columns) 
        self.data=data
        
        print("Features are extracted.")
        print()
        print("The number of data is {}".format(data.shape[0]))
        
        return data
    
    
    def train_test_split(self):
        

        tr_data=[]

        for i in range(len(self.folder)):
            for j in range(len(self.tr_file_path_dic[self.folder[i]])):
                path=self.tr_file_path_dic[self.folder[i]][j]
                df = pd.read_csv(path,skiprows=4,usecols=range(1,7))
                df.columns=['a','b','c','d','e','f']
                columns=['a','b','c','d','e','f']
                data=pd.DataFrame()
                for k in range(6):
                    a=df.describe(include='all').loc[['min','max','mean','50%','std','25%','75%']][columns[k]]
                    data=pd.concat([data,a],ignore_index=True)

                tr_data.append(np.array(data).reshape(1,len(data)))

        tr_data=np.array(tr_data).reshape(len(tr_data),len(columns)*7)
        tr_data=pd.DataFrame(tr_data, columns=self.columns) 
        self.tr_data=tr_data
        
        
        ts_data=[]

        for i in range(len(self.folder)):
            for j in range(len(self.ts_file_path_dic[self.folder[i]])):
                path=self.ts_file_path_dic[self.folder[i]][j]
                df = pd.read_csv(path,skiprows=4,usecols=range(1,7))
                df.columns=['a','b','c','d','e','f']
                columns=['a','b','c','d','e','f']
                data=pd.DataFrame()
                for k in range(6):
                    a=df.describe(include='all').loc[['min','max','mean','50%','std','25%','75%']][columns[k]]
                    data=pd.concat([data,a],ignore_index=True)

                ts_data.append(np.array(data).reshape(1,len(data)))

        ts_data=np.array(ts_data).reshape(len(ts_data),len(columns)*7)
        ts_data=pd.DataFrame(ts_data, columns=self.columns) 
        self.ts_data=ts_data
        
        
        print("Splited train and test data.")
        
        return tr_data, ts_data
        
        
        
    def pick_three(self):
        data=self.data
        pick_three_data=data[['min1','max1','mean1',
             'min2','max2','mean2',
             'min3','max3','mean3',
             'min4','max4','mean4',
             'min5','max5','mean5',
             'min6','max6','mean6']]
        
        print("Min, mean, and max are picked.")
        
        return pick_three_data
    
    
    def get_std(self):
        
        col_dic={}
        columns=self.columns
        for i in range(6):
            col_dic[i]= columns[7*(i+1)-7:7*(i+1)]
            

        
        std_list=[]
        for n in range(6):
            std=np.std(self.data[col_dic[n]])
            std_list.append([std[0],std[1],std[2],std[3],std[4],std[5],std[6]])
        std_list=pd.DataFrame(std_list, columns=['min', 'max', 'mean', '50%', 'std', '25%', '75%'],)
        std_list.index=['avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23']
                        
        return std_list
                        
    
    def do_bootstrap(self):
                        
        columns = self.columns
        (a,b)=self.data.shape
        bs_std_list=[]
        for i in range(b):
            bs_std_list.append((bs.bootstrap(np.array(self.data[columns[i]]),stat_func=bs_stats.std, alpha=0.1)))
   
        
        bs_std_list=pd.DataFrame(bs_std_list,columns=['std_value and range'])
        bs_std_list.index = ['std_min1','std_max1','std_mean1','std_50%_1','std_std1','std_25%_1','std_75%_1',
                             'std_min2','std_max2','std_mean2','std_50%_2','std_std2','std_25%_2','std_75%_2',
                             'std_min3','std_max3','std_mean3','std_50%_3','std_std3','std_25%_3','std_75%_3',
                             'std_min4','std_max4','std_mean4','std_50%_4','std_std4','std_25%_4','std_75%_4',
                             'std_min5','std_max5','std_mean5','std_50%_5','std_std5','std_25%_5','std_75%_5',
                             'std_min6','std_max6','std_mean6','std_50%_6','std_std6','std_25%_6','std_75%_6']
                        
        return bs_std_list 

In [11]:
file_path_dic={}
file_path_dic['bending1']=['../data/AReM/bending1/dataset1.csv','../data/AReM/bending1/dataset2.csv','../data/AReM/bending1/dataset3.csv',
                          '../data/AReM/bending1/dataset4.csv','../data/AReM/bending1/dataset5.csv','../data/AReM/bending1/dataset6.csv',
                          '../data/AReM/bending1/dataset7.csv']
file_path_dic['bending2']=['../data/AReM/bending2/dataset1.csv','../data/AReM/bending2/dataset2.csv','../data/AReM/bending2/dataset3.csv',
                          '../data/AReM/bending2/dataset5.csv','../data/AReM/bending2/dataset6.csv']
file_path_dic['cycling']=['../data/AReM/cycling/dataset1.csv','../data/AReM/cycling/dataset2.csv','../data/AReM/cycling/dataset3.csv',
                          '../data/AReM/cycling/dataset4.csv','../data/AReM/cycling/dataset5.csv','../data/AReM/cycling/dataset6.csv','../data/AReM/cycling/dataset7.csv',
                         '../data/AReM/cycling/dataset8.csv','../data/AReM/cycling/dataset9.csv','../data/AReM/cycling/dataset10.csv','../data/AReM/cycling/dataset11.csv','../data/AReM/cycling/dataset12.csv',
                         '../data/AReM/cycling/dataset13.csv','../data/AReM/cycling/dataset14.csv','../data/AReM/cycling/dataset15.csv']
file_path_dic['lying']=['../data/AReM/lying/dataset1.csv','../data/AReM/lying/dataset2.csv','../data/AReM/lying/dataset3.csv',
                          '../data/AReM/lying/dataset4.csv','../data/AReM/lying/dataset5.csv','../data/AReM/lying/dataset6.csv','../data/AReM/lying/dataset7.csv',
                         '../data/AReM/lying/dataset8.csv','../data/AReM/lying/dataset9.csv','../data/AReM/lying/dataset10.csv','../data/AReM/lying/dataset11.csv','../data/AReM/lying/dataset12.csv',
                         '../data/AReM/lying/dataset13.csv','../data/AReM/lying/dataset14.csv','../data/AReM/lying/dataset15.csv']
file_path_dic['sitting']=['../data/AReM/sitting/dataset1.csv','../data/AReM/sitting/dataset2.csv','../data/AReM/sitting/dataset3.csv',
                          '../data/AReM/sitting/dataset4.csv','../data/AReM/sitting/dataset5.csv','../data/AReM/sitting/dataset6.csv','../data/AReM/sitting/dataset7.csv',
                         '../data/AReM/sitting/dataset8.csv','../data/AReM/sitting/dataset9.csv','../data/AReM/sitting/dataset10.csv','../data/AReM/sitting/dataset11.csv','../data/AReM/sitting/dataset12.csv',
                         '../data/AReM/sitting/dataset13.csv','../data/AReM/sitting/dataset14.csv','../data/AReM/sitting/dataset15.csv']
file_path_dic['standing']=['../data/AReM/standing/dataset1.csv','../data/AReM/standing/dataset2.csv','../data/AReM/standing/dataset3.csv',
                          '../data/AReM/standing/dataset4.csv','../data/AReM/standing/dataset5.csv','../data/AReM/standing/dataset6.csv','../data/AReM/standing/dataset7.csv',
                         '../data/AReM/standing/dataset8.csv','../data/AReM/standing/dataset9.csv','../data/AReM/standing/dataset10.csv','../data/AReM/standing/dataset11.csv','../data/AReM/standing/dataset12.csv',
                         '../data/AReM/standing/dataset13.csv','../data/AReM/standing/dataset14.csv','../data/AReM/standing/dataset15.csv']
file_path_dic['walking']=['../data/AReM/walking/dataset1.csv','../data/AReM/walking/dataset2.csv','../data/AReM/walking/dataset3.csv',
                          '../data/AReM/walking/dataset4.csv','../data/AReM/walking/dataset5.csv','../data/AReM/walking/dataset6.csv','../data/AReM/walking/dataset7.csv',
                         '../data/AReM/walking/dataset8.csv','../data/AReM/walking/dataset9.csv','../data/AReM/walking/dataset10.csv','../data/AReM/walking/dataset11.csv','../data/AReM/walking/dataset12.csv',
                         '../data/AReM/walking/dataset13.csv','../data/AReM/walking/dataset14.csv','../data/AReM/walking/dataset15.csv']

tr_file_path_dic={}
tr_file_path_dic['bending1']=['../data/AReM/bending1/dataset3.csv','../data/AReM/bending1/dataset4.csv','../data/AReM/bending1/dataset5.csv','../data/AReM/bending1/dataset6.csv',
                                  '../data/AReM/bending1/dataset7.csv']
tr_file_path_dic['bending2']=['../data/AReM/bending2/dataset3.csv','../data/AReM/bending2/dataset4.csv',
                                  '../data/AReM/bending2/dataset5.csv','../data/AReM/bending2/dataset6.csv']
tr_file_path_dic['cycling']=['../data/AReM/cycling/dataset4.csv','../data/AReM/cycling/dataset5.csv','../data/AReM/cycling/dataset6.csv','../data/AReM/cycling/dataset7.csv',
                                 '../data/AReM/cycling/dataset8.csv','../data/AReM/cycling/dataset9.csv','../data/AReM/cycling/dataset10.csv','../data/AReM/cycling/dataset11.csv','../data/AReM/cycling/dataset12.csv',
                                 '../data/AReM/cycling/dataset13.csv','../data/AReM/cycling/dataset14.csv','../data/AReM/cycling/dataset15.csv']
tr_file_path_dic['lying']=['../data/AReM/lying/dataset4.csv','../data/AReM/lying/dataset5.csv','../data/AReM/lying/dataset6.csv','../data/AReM/lying/dataset7.csv',
                                 '../data/AReM/lying/dataset8.csv','../data/AReM/lying/dataset9.csv','../data/AReM/lying/dataset10.csv','../data/AReM/lying/dataset11.csv','../data/AReM/lying/dataset12.csv',
                                 '../data/AReM/lying/dataset13.csv','../data/AReM/lying/dataset14.csv','../data/AReM/lying/dataset15.csv']
tr_file_path_dic['sitting']=['../data/AReM/sitting/dataset4.csv','../data/AReM/sitting/dataset5.csv','../data/AReM/sitting/dataset6.csv','../data/AReM/sitting/dataset7.csv',
                                 '../data/AReM/sitting/dataset8.csv','../data/AReM/sitting/dataset9.csv','../data/AReM/sitting/dataset10.csv','../data/AReM/sitting/dataset11.csv','../data/AReM/sitting/dataset12.csv',
                                 '../data/AReM/sitting/dataset13.csv','../data/AReM/sitting/dataset14.csv','../data/AReM/sitting/dataset15.csv']
tr_file_path_dic['standing']=['../data/AReM/standing/dataset4.csv','../data/AReM/standing/dataset5.csv','../data/AReM/standing/dataset6.csv','../data/AReM/standing/dataset7.csv',
                                 '../data/AReM/standing/dataset8.csv','../data/AReM/standing/dataset9.csv','../data/AReM/standing/dataset10.csv','../data/AReM/standing/dataset11.csv','../data/AReM/standing/dataset12.csv',
                                 '../data/AReM/standing/dataset13.csv','../data/AReM/standing/dataset14.csv','../data/AReM/standing/dataset15.csv']
tr_file_path_dic['walking']=['../data/AReM/walking/dataset4.csv','../data/AReM/walking/dataset5.csv','../data/AReM/walking/dataset6.csv','../data/AReM/walking/dataset7.csv',
                                 '../data/AReM/walking/dataset8.csv','../data/AReM/walking/dataset9.csv','../data/AReM/walking/dataset10.csv','../data/AReM/walking/dataset11.csv','../data/AReM/walking/dataset12.csv',
                                 '../data/AReM/walking/dataset13.csv','../data/AReM/walking/dataset14.csv','../data/AReM/walking/dataset15.csv']

ts_file_path_dic={}
ts_file_path_dic['bending1']=['../data/AReM/bending1/dataset1.csv','../data/AReM/bending1/dataset2.csv']
ts_file_path_dic['bending2']=['../data/AReM/bending2/dataset1.csv','../data/AReM/bending2/dataset2.csv']                   
ts_file_path_dic['cycling']=['../data/AReM/cycling/dataset1.csv','../data/AReM/cycling/dataset2.csv','../data/AReM/cycling/dataset3.csv']                   
ts_file_path_dic['lying']=['../data/AReM/lying/dataset1.csv','../data/AReM/lying/dataset2.csv','../data/AReM/lying/dataset3.csv']                
ts_file_path_dic['sitting']=['../data/AReM/sitting/dataset1.csv','../data/AReM/sitting/dataset2.csv','../data/AReM/sitting/dataset3.csv']            
ts_file_path_dic['standing']=['../data/AReM/standing/dataset1.csv','../data/AReM/standing/dataset2.csv','../data/AReM/standing/dataset3.csv']
ts_file_path_dic['walking']=['../data/AReM/walking/dataset1.csv','../data/AReM/walking/dataset2.csv','../data/AReM/walking/dataset3.csv']
                               

TSC=TimeSeriesClassification(file_path_dic,tr_file_path_dic,ts_file_path_dic)

In [ ]:
#1.b
tr_data, ts_data = TSC.train_test_split()

In [ ]:
#1.c.i

print("""

Usually, in time-series classification problem, \n
means, variance, maximum value, minimum value, quantile, auto-correlation and variance are considered.

""")

In [ ]:
#1.c.ii
data = TSC.feature_extraction()

In [ ]:
#1.c.iii
std_list = TSC.get_std()
std_list

In [ ]:
#1.c.iii
bs_std_list=TSC.do_bootstrap()


In [ ]:
#1.c.iv

pick_three_data=TSC.pick_three()


print("""

I picked min, mean, and max for each variable.

""")